In [35]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from string import punctuation
punctuation = list(punctuation)
from razdel import tokenize
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
import pymorphy2
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Максем\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Максем\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Максем\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
df = pd.read_csv(r"C:\Users\Максем\Downloads\archive (2)\spam_or_not_spam.csv")
df.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [3]:
df['email'].fillna(method = 'ffill', inplace = True)
df['email'].isnull().sum()

0

In [4]:
stopwords = nltk.corpus.stopwords.words('english')
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
def normalize_text(s):
   #Очистка от html
    soup = BeautifulSoup(s, 'html.parser')
    soup.get_text()
   #Удаление скрипта
    for data in soup(['style', 'script']):
      data.decompose()
    script_out = ' '.join(soup.stripped_strings)
  #Токенизация
    tokens = word_tokenize(s)
  # Удаляем пунктуацию
    tokens_without_punct = [i for i in tokens if i not in punctuation]
    low_tokens = [i.lower() for i in tokens_without_punct]
  # удаляем стоп-слова из нашего текста
    stopwords = nltk.corpus.stopwords.words('english')
    words_without_stop = [i for i in low_tokens if i not in stopwords]
  # Лемматизация
    lemmatizer = nltk.WordNetLemmatizer()
    lemms = [lemmatizer.lemmatize(word) for word in words_without_stop]
  # Стемминг
    ps = PorterStemmer()
    stems = [ps.stem(i) for i in lemms]
  # Вывод значения в строке
    total=''
    for el in stems:
      total+=el
      total+=' '
    return total

In [6]:
df['normalized'] = df['email'].apply(normalize_text)

In [7]:
df.head()

,email,label,normalized
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0,date wed number aug number number number numbe...
1,martin a posted tassos papadopoulos the greek ...,0,martin post tasso papadopoulo greek sculptor b...
2,man threatens explosion in moscow thursday aug...,0,man threaten explos moscow thursday august num...
3,klez the virus that won t die already the most...,0,klez viru die alreadi prolif viru ever klez co...
4,in adding cream to spaghetti carbonara which ...,0,ad cream spaghetti carbonara effect pasta make...


In [8]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['normalized'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_df.head()

,__,___,____,_____,______,_______,________,_________,__________,______________,...,허락없이,헤어디자이너,현재,호황을,홈쇼핑의,확실한,활황을,훨씬,힘입어,ｉt的技
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(df['normalized'])
count_df = pd.DataFrame(data=count_matrix.toarray(), columns=count_vectorizer.get_feature_names_out())
count_df.head()

,__,___,____,_____,______,_______,________,_________,__________,______________,...,허락없이,헤어디자이너,현재,호황을,홈쇼핑의,확실한,활황을,훨씬,힘입어,ｉt的技
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
X = tfidf_df
y = df['label']

In [11]:
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.model_selection import cross_validate

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
t_grid_search = GridSearchCV(DecisionTreeClassifier(),
                               [{'max_depth': [5, 10, 15],'min_samples_leaf': [1, 2, 4]}],
                               cv=5,
                               verbose=10)

In [14]:
t_grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START max_depth=5, min_samples_leaf=1.............................
[CV 1/5; 1/9] END max_depth=5, min_samples_leaf=1;, score=0.944 total time=   2.8s
[CV 2/5; 1/9] START max_depth=5, min_samples_leaf=1.............................
[CV 2/5; 1/9] END max_depth=5, min_samples_leaf=1;, score=0.944 total time=   2.9s
[CV 3/5; 1/9] START max_depth=5, min_samples_leaf=1.............................
[CV 3/5; 1/9] END max_depth=5, min_samples_leaf=1;, score=0.958 total time=   3.1s
[CV 4/5; 1/9] START max_depth=5, min_samples_leaf=1.............................
[CV 4/5; 1/9] END max_depth=5, min_samples_leaf=1;, score=0.971 total time=   2.9s
[CV 5/5; 1/9] START max_depth=5, min_samples_leaf=1.............................
[CV 5/5; 1/9] END max_depth=5, min_samples_leaf=1;, score=0.958 total time=   2.4s
[CV 1/5; 2/9] START max_depth=5, min_samples_leaf=2.............................
[CV 1/5; 2/9] END max_depth=5, min_samp

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid=[{'max_depth': [5, 10, 15],
                          'min_samples_leaf': [1, 2, 4]}],
             verbose=10)

In [15]:
print(t_grid_search.best_params_)
print(t_grid_search.best_score_)
print(t_grid_search.best_estimator_)


{'max_depth': 15, 'min_samples_leaf': 1}
0.9595833333333333
DecisionTreeClassifier(max_depth=15)


In [16]:
C = count_df
y = df['label']

In [17]:
C_train, C_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
cvec_grid_search = GridSearchCV(DecisionTreeClassifier(),
                               [{'max_depth': [5, 10, 15],'min_samples_leaf': [1, 2, 4]}],
                               cv=5,
                               verbose=10)

In [19]:
cvec_grid_search.fit(C_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START max_depth=5, min_samples_leaf=1.............................
[CV 1/5; 1/9] END max_depth=5, min_samples_leaf=1;, score=0.946 total time=   2.8s
[CV 2/5; 1/9] START max_depth=5, min_samples_leaf=1.............................
[CV 2/5; 1/9] END max_depth=5, min_samples_leaf=1;, score=0.946 total time=   2.7s
[CV 3/5; 1/9] START max_depth=5, min_samples_leaf=1.............................
[CV 3/5; 1/9] END max_depth=5, min_samples_leaf=1;, score=0.958 total time=   2.8s
[CV 4/5; 1/9] START max_depth=5, min_samples_leaf=1.............................
[CV 4/5; 1/9] END max_depth=5, min_samples_leaf=1;, score=0.971 total time=   2.7s
[CV 5/5; 1/9] START max_depth=5, min_samples_leaf=1.............................
[CV 5/5; 1/9] END max_depth=5, min_samples_leaf=1;, score=0.958 total time=   2.4s
[CV 1/5; 2/9] START max_depth=5, min_samples_leaf=2.............................
[CV 1/5; 2/9] END max_depth=5, min_samp

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid=[{'max_depth': [5, 10, 15],
                          'min_samples_leaf': [1, 2, 4]}],
             verbose=10)

In [21]:
print(cvec_grid_search.best_params_)
print(cvec_grid_search.best_score_)
print(cvec_grid_search.best_estimator_)

{'max_depth': 15, 'min_samples_leaf': 1}
0.9591666666666667
DecisionTreeClassifier(max_depth=15)


В результате сравнения лучшего скора кросс - валидации для дерева решений, точность выше у векторизации текста при помощи TF - IDF

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
l_grid_search = GridSearchCV(LogisticRegression(),
                               [{'penalty': ['l1', 'l2', 'elasticnet'], 'C': [0.1, 1, 10, 100]  }],
                               cv=5,
                               verbose=10)

In [24]:
l_grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5; 1/12] START C=0.1, penalty=l1..........................................
[CV 1/5; 1/12] END .............C=0.1, penalty=l1;, score=nan total time=   0.2s
[CV 2/5; 1/12] START C=0.1, penalty=l1..........................................
[CV 2/5; 1/12] END .............C=0.1, penalty=l1;, score=nan total time=   0.1s
[CV 3/5; 1/12] START C=0.1, penalty=l1..........................................
[CV 3/5; 1/12] END .............C=0.1, penalty=l1;, score=nan total time=   0.1s
[CV 4/5; 1/12] START C=0.1, penalty=l1..........................................
[CV 4/5; 1/12] END .............C=0.1, penalty=l1;, score=nan total time=   0.1s
[CV 5/5; 1/12] START C=0.1, penalty=l1..........................................
[CV 5/5; 1/12] END .............C=0.1, penalty=l1;, score=nan total time=   0.1s
[CV 1/5; 2/12] START C=0.1, penalty=l2..........................................
[CV 1/5; 2/12] END ...........C=0.1, penalty=l2;

c:\Users\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\Users\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
ValueError: Solver lbfgs sup

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid=[{'C': [0.1, 1, 10, 100],
                          'penalty': ['l1', 'l2', 'elasticnet']}],
             verbose=10)

In [25]:
print(l_grid_search.best_params_)
print(l_grid_search.best_score_)
print(l_grid_search.best_estimator_)

{'C': 100, 'penalty': 'l2'}
0.9870833333333333
LogisticRegression(C=100)


In [30]:
C = count_df
y = df['label']

In [31]:
C_train, C_test, y_train, y_test = train_test_split(C, y, test_size=0.2, random_state=42)

In [32]:
lvec_grid_search = GridSearchCV(LogisticRegression(),
                               [{'penalty': ['l1', 'l2', 'elasticnet'], 'C': [0.1, 1, 10, 100]  }],
                               cv=5,
                               verbose=10)

In [33]:
lvec_grid_search.fit(C_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5; 1/12] START C=0.1, penalty=l1..........................................
[CV 1/5; 1/12] END .............C=0.1, penalty=l1;, score=nan total time=   0.1s
[CV 2/5; 1/12] START C=0.1, penalty=l1..........................................
[CV 2/5; 1/12] END .............C=0.1, penalty=l1;, score=nan total time=   0.1s
[CV 3/5; 1/12] START C=0.1, penalty=l1..........................................
[CV 3/5; 1/12] END .............C=0.1, penalty=l1;, score=nan total time=   0.1s
[CV 4/5; 1/12] START C=0.1, penalty=l1..........................................
[CV 4/5; 1/12] END .............C=0.1, penalty=l1;, score=nan total time=   0.1s
[CV 5/5; 1/12] START C=0.1, penalty=l1..........................................
[CV 5/5; 1/12] END .............C=0.1, penalty=l1;, score=nan total time=   0.1s
[CV 1/5; 2/12] START C=0.1, penalty=l2..........................................
[CV 1/5; 2/12] END ...........C=0.1, penalty=l2;

c:\Users\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 2/5; 5/12] END .............C=1, penalty=l2;, score=0.990 total time=   5.4s
[CV 3/5; 5/12] START C=1, penalty=l2............................................
[CV 3/5; 5/12] END .............C=1, penalty=l2;, score=0.992 total time=   5.2s
[CV 4/5; 5/12] START C=1, penalty=l2............................................
[CV 4/5; 5/12] END .............C=1, penalty=l2;, score=0.983 total time=   5.3s
[CV 5/5; 5/12] START C=1, penalty=l2............................................
[CV 5/5; 5/12] END .............C=1, penalty=l2;, score=0.994 total time=   4.8s
[CV 1/5; 6/12] START C=1, penalty=elasticnet....................................
[CV 1/5; 6/12] END .......C=1, penalty=elasticnet;, score=nan total time=   0.1s
[CV 2/5; 6/12] START C=1, penalty=elasticnet....................................
[CV 2/5; 6/12] END .......C=1, penalty=elasticnet;, score=nan total time=   0.1s
[CV 3/5; 6/12] START C=1, penalty=elasticnet....................................
[CV 3/5; 6/12] END .......C=

c:\Users\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 2/5; 8/12] END ............C=10, penalty=l2;, score=0.985 total time=   6.2s
[CV 3/5; 8/12] START C=10, penalty=l2...........................................


...

[CV 4/5; 11/12] END ..........C=100, penalty=l2;, score=0.988 total time=   6.9s
[CV 5/5; 11/12] START C=100, penalty=l2.........................................


c:\Users\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 5/5; 11/12] END ..........C=100, penalty=l2;, score=0.985 total time=   6.7s
[CV 1/5; 12/12] START C=100, penalty=elasticnet.................................
[CV 1/5; 12/12] END ....C=100, penalty=elasticnet;, score=nan total time=   0.1s
[CV 2/5; 12/12] START C=100, penalty=elasticnet.................................
[CV 2/5; 12/12] END ....C=100, penalty=elasticnet;, score=nan total time=   0.1s
[CV 3/5; 12/12] START C=100, penalty=elasticnet.................................
[CV 3/5; 12/12] END ....C=100, penalty=elasticnet;, score=nan total time=   0.1s
[CV 4/5; 12/12] START C=100, penalty=elasticnet.................................
[CV 4/5; 12/12] END ....C=100, penalty=elasticnet;, score=nan total time=   0.1s
[CV 5/5; 12/12] START C=100, penalty=elasticnet.................................
[CV 5/5; 12/12] END ....C=100, penalty=elasticnet;, score=nan total time=   0.1s


c:\Users\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\Users\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
ValueError: Solver lbfgs sup

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid=[{'C': [0.1, 1, 10, 100],
                          'penalty': ['l1', 'l2', 'elasticnet']}],
             verbose=10)

In [34]:
print(lvec_grid_search.best_params_)
print(lvec_grid_search.best_score_)
print(lvec_grid_search.best_estimator_)

{'C': 1, 'penalty': 'l2'}
0.9879166666666667
LogisticRegression(C=1)


Для модели логистической регрессии точность лучше у count_vectorizer

In [36]:
X = tfidf_df
y = df['label']

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
naivetf_grid_search = GridSearchCV(GaussianNB(),
                               [{'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]}],
                               cv=5,
                               verbose=10)

In [40]:
naivetf_grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5; 1/4] START var_smoothing=1e-09.........................................
[CV 1/5; 1/4] END ..........var_smoothing=1e-09;, score=0.952 total time=   1.7s
[CV 2/5; 1/4] START var_smoothing=1e-09.........................................
[CV 2/5; 1/4] END ..........var_smoothing=1e-09;, score=0.931 total time=   1.5s
[CV 3/5; 1/4] START var_smoothing=1e-09.........................................
[CV 3/5; 1/4] END ..........var_smoothing=1e-09;, score=0.940 total time=   1.6s
[CV 4/5; 1/4] START var_smoothing=1e-09.........................................
[CV 4/5; 1/4] END ..........var_smoothing=1e-09;, score=0.929 total time=   1.5s
[CV 5/5; 1/4] START var_smoothing=1e-09.........................................
[CV 5/5; 1/4] END ..........var_smoothing=1e-09;, score=0.948 total time=   1.5s
[CV 1/5; 2/4] START var_smoothing=1e-08.........................................
[CV 1/5; 2/4] END ..........var_smoothing=1e-08;,

GridSearchCV(cv=5, estimator=GaussianNB(),
             param_grid=[{'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06]}],
             verbose=10)

In [41]:
print(naivetf_grid_search.best_params_)
print(naivetf_grid_search.best_score_)
print(naivetf_grid_search.best_estimator_)

{'var_smoothing': 1e-06}
0.94625
GaussianNB(var_smoothing=1e-06)


In [42]:
C = count_df
y = df['label']

In [43]:
C_train, C_test, y_train, y_test = train_test_split(C, y, test_size=0.2, random_state=42)

In [46]:
naivevec_grid_search = GridSearchCV(GaussianNB(),
                               [{'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]}],
                               cv=5,
                               verbose=10)

In [47]:
naivevec_grid_search.fit(C_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5; 1/4] START var_smoothing=1e-09.........................................
[CV 1/5; 1/4] END ..........var_smoothing=1e-09;, score=0.960 total time=   1.7s
[CV 2/5; 1/4] START var_smoothing=1e-09.........................................
[CV 2/5; 1/4] END ..........var_smoothing=1e-09;, score=0.931 total time=   1.7s
[CV 3/5; 1/4] START var_smoothing=1e-09.........................................
[CV 3/5; 1/4] END ..........var_smoothing=1e-09;, score=0.944 total time=   1.6s
[CV 4/5; 1/4] START var_smoothing=1e-09.........................................
[CV 4/5; 1/4] END ..........var_smoothing=1e-09;, score=0.921 total time=   1.7s
[CV 5/5; 1/4] START var_smoothing=1e-09.........................................
[CV 5/5; 1/4] END ..........var_smoothing=1e-09;, score=0.948 total time=   1.6s
[CV 1/5; 2/4] START var_smoothing=1e-08.........................................
[CV 1/5; 2/4] END ..........var_smoothing=1e-08;,

GridSearchCV(cv=5, estimator=GaussianNB(),
             param_grid=[{'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06]}],
             verbose=10)

In [48]:
print(naivevec_grid_search.best_params_)
print(naivevec_grid_search.best_score_)
print(naivevec_grid_search.best_estimator_)

{'var_smoothing': 1e-07}
0.9425000000000001
GaussianNB(var_smoothing=1e-07)


В результате сравнения для Naive Bayes точность выше при TF - IDF